# Just for testing random things

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import numpy as np
import jax
import jax.numpy as jnp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from functools import partial
import os
import sys
import umap
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)


from synbio_morpher.utils.common.setup import construct_circuit_from_cfg, prepare_config
from synbio_morpher.utils.misc.string_handling import string_to_tuple_list
from synbio_morpher.utils.results.analytics.naming import get_true_interaction_cols
from synbio_morpher.srv.parameter_prediction.simulator import SIMULATOR_UNITS
from synbio_morpher.utils.misc.numerical import make_symmetrical_matrix_from_sequence, triangular_sequence
from synbio_morpher.utils.misc.type_handling import flatten_listlike, get_first_elements
from synbio_morpher.utils.misc.string_handling import prettify_keys_for_label
from synbio_morpher.utils.results.analytics.naming import get_analytics_types_all
from synbio_morpher.utils.common.testing.shared import CONFIG
from synbio_morpher.utils.parameter_inference.interpolation_grid import create_parameter_range
from explanations.data_enhancing import enhance_data


In [105]:
import numpy as np
import scipy.integrate
import scipy.optimize
import matplotlib

# import be150

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [106]:
parameter_based_simulation = {
        "interaction_min": 0,
        "interaction_max": 1,
        "step_size": 0.1,
        "log_scale": False
    }

interaction_strengths = create_parameter_range(parameter_based_simulation)
num_species = 3
num_unique_interactions = int(np.sum(np.triu(np.ones((num_species, num_species)))))
analytic_types = get_analytics_types_all()
size_interaction_array = np.size(interaction_strengths)


interpolation_grid.py:create_parameter_range():35: create_parameter_range INFO
interpolation_grid.py:create_parameter_range():36: {'interaction_min': 0, 'interaction_max': 1, 'step_size': 0.1, 'log_scale': False} INFO


In [107]:
def idx_sampler(i, size_interaction_array, unique_interaction):
    return jnp.mod(i / jnp.power(size_interaction_array, unique_interaction), size_interaction_array).astype(int)

unique_interaction = np.arange(num_unique_interactions)
idx_sampler = partial(idx_sampler, size_interaction_array=size_interaction_array, unique_interaction=unique_interaction)
    
starting_iteration = 0
end_iteration = 10

i = np.arange(1500, 1605)[:, None]

interaction_strength_choices = jax.vmap(idx_sampler)(i)


In [108]:
flat_triangle = interaction_strengths[list(interaction_strength_choices)]


In [109]:
interaction_matrices = [
    make_symmetrical_matrix_from_sequence(f, side_length=num_species) for f in flat_triangle]

circuits = [None] * len(flat_triangle)
for i, interaction_matrix in enumerate(interaction_matrices):
    cfg = {"interactions": {
        "interactions_matrix": interaction_matrix,
        "interactions_units": SIMULATOR_UNITS['IntaRNA']['rate']}
    }
    circuits[i] = construct_circuit_from_cfg(
        extra_configs=cfg, config_file=config)

NameError: name 'config' is not defined